In [18]:
#set some names, because why not

#Path to the repo for this project
repoPath='/Users/plab/Documents/ipynb/'

#file name of standard map of the world
firstMapName='Earthmap1000x500.jpg'

#file name of grayscale map of the world
grayscaleMapName='World_map_blank_without_borders.svg.png'

#file name of world map with "graticule" (lines)
#see https://en.wikipedia.org/wiki/Geographic_coordinate_system for more info
linedMapName='Equirectangular_projection_SW.jpg'

#file name of political world map
politicalMapName='2000px-Dünya.svg.png'

In [19]:
#loading image processing and manipulation package Pillow
# https://pillow.readthedocs.io/en/stable/
import PIL
from PIL import Image
import os
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import numpy as np

politicalMapPath=os.path.join(repoPath,'images',politicalMapName)   

print(politicalMapPath)
politicalMap= Image.open(politicalMapName)

#in order to display in jupyter, some trickery is necessary
%matplotlib inline
imshow(np.asarray(politicalMap))
fig = plt.gcf()
fig.set_size_inches(15, 30)

/Users/plab/Documents/ipynb/images/2000px-Dünya.svg.png


FileNotFoundError: [Errno 2] No such file or directory: '2000px-Dünya.svg.png'